<a href="https://colab.research.google.com/github/NicolasFerre/ColabFerre/blob/main/Consolidaci%C3%B3n_de_pedidos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalaciones previas

In [148]:
#!pip install xlsxwriter
#!pip install python-calamine
#!pip install fastexcel

# Librerías necesarias

In [149]:
import polars as ps
import pandas as pd
import numpy as np

# Reportes de Amazon
Aquí se definen cuantos reportes y transacciones se van a procesar

In [150]:
loop = True
while loop:
  try:
    reportes_cantidad = input("Cantidad de reportes: ")
    reportes_cantidad = int(reportes_cantidad)
    loop = False
  except:
    print("Ingresa un número válido")

Cantidad de reportes: 3


In [151]:
transacciones_cantidad = input("Cantidad de transacciones: ")
transacciones_cantidad = int(transacciones_cantidad)

Cantidad de transacciones: 57


# Leyendo Reportes

Si solo se va a leer un reporte, este deberá ser guardado como "Reporte". Si no, los reportes deberán ser guardados de forma seriada, es decir, "Reporte1", "Reporte2", "Reporte3", etc.

El orden no debería importar

In [152]:
archivo = ''
archivos = []
if reportes_cantidad == 1:
    archivo = "Reportes/Reporte.txt"
else:
  for i in range(reportes_cantidad):
      archivos.append((f"Reportes/Reporte{i+1}.txt"))

In [153]:
if archivo == '':
  reportes = ps.read_csv(archivos[0], separator="\t")
else:
  reportes = ps.read_csv(archivo, separator="\t")

In [154]:
if len(archivos) > 1:
  for i in range(1, reportes_cantidad):
      reportes = reportes.vstack(ps.read_csv(archivos[i], separator="\t", schema=reportes.schema, truncate_ragged_lines=True))
  print(f"Uniendo {reportes_cantidad} reportes")


Uniendo 3 reportes


# Leyendo Transacciones

Si solo se va a leer una transacción, esta deberá ser guardada como "Transacciones". Si no, deberán ser guardadas de forma seriada, como los reportes.

***Nota: Los reportes deberán ser pasados primero por la herramienta de limpieza de transacciones. Si no, generarán problemas***

In [155]:
archivo = ''
archivos = []
if transacciones_cantidad == 1:
    archivo = "Transacciones/Transacciones.csv"
else:
  for i in range(transacciones_cantidad):
      archivos.append((f"Transacciones/Transacciones{i+1}.csv"))

In [156]:
if archivo == '':
  transacciones = ps.read_csv(archivos[0], separator=",")
else:
  transacciones = ps.read_csv(archivo, separator=",")

In [157]:
if len(archivos) > 1:
  for i in range(1, transacciones_cantidad):
      transacciones = transacciones.vstack(ps.read_csv(archivos[i], separator = ',', schema=transacciones.schema, truncate_ragged_lines=True))

Renombramos la columna de Número de pedido a amazon-order-id, para hacer la unión luego con el reporte de pedidos

In [158]:
transacciones = transacciones.rename({'Número de pedido':'amazon-order-id'})

In [159]:
dfr = reportes.join(transacciones, on='amazon-order-id', how='left', suffix='_transacciones')

In [160]:
formato_num = {ps.Int64:"0", ps.Float64:"0.00"}

In [161]:
dft = transacciones.join(reportes, on='amazon-order-id', how='left', suffix='_reportes')

# Consolidando reporte de envíos

In [162]:
archivo_envios = 'Envios enero-julio.xlsx'
envios_1 = pd.read_excel(archivo_envios, sheet_name = 0, header=3)
envios_2 = pd.read_excel(archivo_envios, sheet_name = 1, header=3)
reporte_envios = pd.concat([envios_1, envios_2])
reporte_envios = reporte_envios.reset_index(drop=True)
reporte_envios.to_excel("Envios.xlsx", index=False)

In [163]:
envios = ps.read_excel("Envios.xlsx", sheet_id=1)

In [164]:
envios = envios.with_columns(ps.col('PROTECCION ($)').fill_null(0))
envios = envios.with_columns(ps.col('PROTECCION ($)').cast(ps.Float32))
envios.schema

Schema([('NUMERO ORDEN', String),
        ('GUÍA', String),
        ('PAQUETERIA', String),
        ('SERVICIO', String),
        ('FECHA DE CREACION', Date),
        ('NOMBRE DESTINO', String),
        ('CP DESTINO', Int64),
        ('VALOR GUIA ($)', Float64),
        ('ZONA EXTENDIDA ($)', Float64),
        ('EXTRAGRANDE ($)', Int64),
        ('SOBREPESOS ($)', Int64),
        ('TOTAL GUIA ($)', Float64),
        ('ESTATUS RASTREO', String),
        ('PROTECCION ($)', Float32)])

# Juntando el reporte de envíos y las transacciones

In [165]:
envios = envios.rename({'NUMERO ORDEN':'amazon-order-id'})
envios = envios.select(['amazon-order-id', 'VALOR GUIA ($)', 'ZONA EXTENDIDA ($)', 'EXTRAGRANDE ($)', 'SOBREPESOS ($)', 'PROTECCION ($)', 'TOTAL GUIA ($)'])
dfr = dfr.join(envios, on='amazon-order-id', how='left')

In [166]:
dfr.write_excel('Reporte de envios1.xlsx')

In [167]:
dft = dft.join(envios, on='amazon-order-id', how='left')
dft.write_excel('Reporte de envios2.xlsx')

In [169]:
dft.filter(ps.col('amazon-order-id') != '---')

Fecha,Tipo de transacción,amazon-order-id,Detalles del producto,Total de cargos del producto,Total de reembolsos promocionales,Tarifas de Amazon,Otros,Total (MXN),merchant-order-id,purchase-date,last-updated-date,order-status,fulfillment-channel,sales-channel,order-channel,url,ship-service-level,product-name,sku,asin,item-status,quantity,currency,item-price,item-tax,shipping-price,shipping-tax,gift-wrap-price,gift-wrap-tax,item-promotion-discount,ship-promotion-discount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,payment-method-details,item-extensions-data,is-business-order,purchase-order-number,price-designation,customized-url,customized-page,is-replacement-order,is-exchange-order,original-order-id,is-transparency,is-buyer-requested-cancellation,buyer-requested-cancel-reason,buyer-identification-number,buyer-identification-type,VALOR GUIA ($),ZONA EXTENDIDA ($),EXTRAGRANDE ($),SOBREPESOS ($),PROTECCION ($),TOTAL GUIA ($)
str,str,str,str,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,f64,f64,f64,str,str,str,str,f64,str,str,i64,str,str,str,str,bool,str,str,str,str,bool,bool,str,bool,bool,str,str,str,f64,f64,i64,i64,f32,f64
"""3/5/2025""","""Pago del pedido""","""701-5579061-5906663""","""Truper ESE-20, Escalera de ext…",3103.45,0.0,-648.0,496.55,2952.0,null,"""2025-05-03T18:53:57+00:00""","""2025-05-03T23:02:27+00:00""","""Shipped""","""Merchant""","""Amazon.com.mx""","""WebsiteOrderChannel""",null,"""Standard""","""Truper ESE-20, Escalera de ext…","""16027""","""B013R4N3M2""","""Shipped""",1,"""MXN""",3600.0,496.55,null,null,null,null,null,null,"""GUADALAJARA""","""JALISCO""",44330,"""MX""",null,"""CreditCard""",null,false,null,null,null,null,false,false,null,false,false,null,null,null,376.0,0.0,0,0,0.0,376.0
"""3/5/2025""","""Servicios de envío adquiridos …","""702-8036310-0469852""","""Facturación""",-180.32,0.0,0.0,-159.79,-340.11,null,"""2025-05-03T21:09:18+00:00""","""2025-05-03T22:53:24+00:00""","""Shipped""","""Merchant""","""Amazon.com.mx""","""WebsiteOrderChannel""",null,"""Standard""","""Truper ESE-24, Escalera de ext…","""ESE-24""","""B013R4N818""","""Shipped""",1,"""MXN""",4250.0,586.21,null,null,null,null,null,null,"""SANCTORUM DE LAZARO CARDENAS""","""TLAXCALA""",90230,"""MX""",null,"""CreditCard""",null,false,null,null,null,null,false,false,null,false,false,null,null,null,null,null,null,null,null,null
"""3/5/2025""","""Pago del pedido""","""702-8036310-0469852""","""Truper ESE-24, Escalera de ext…",3663.79,0.0,-765.0,586.21,3485.0,null,"""2025-05-03T21:09:18+00:00""","""2025-05-03T22:53:24+00:00""","""Shipped""","""Merchant""","""Amazon.com.mx""","""WebsiteOrderChannel""",null,"""Standard""","""Truper ESE-24, Escalera de ext…","""ESE-24""","""B013R4N818""","""Shipped""",1,"""MXN""",4250.0,586.21,null,null,null,null,null,null,"""SANCTORUM DE LAZARO CARDENAS""","""TLAXCALA""",90230,"""MX""",null,"""CreditCard""",null,false,null,null,null,null,false,false,null,false,false,null,null,null,null,null,null,null,null,null
"""3/5/2025""","""Pago del pedido""","""702-6243348-7873846""","""Truper EST-29, Escalera tipo t…",2715.52,0.0,-567.01,434.48,2582.99,null,"""2025-05-02T23:32:50+00:00""","""2025-05-03T18:34:57+00:00""","""Shipped""","""Merchant""","""Amazon.com.mx""","""WebsiteOrderChannel""",null,"""Standard""","""Truper EST-29, Escalera tipo t…","""0F-ZVA0-7RNO""","""B013QH5BBG""","""Shipped""",1,"""MXN""",3150.0,434.48,null,null,null,null,null,null,"""HERMOSILLO""","""SONORA""",83249,"""MX""",null,"""Installments""",null,false,null,null,null,null,false,false,null,false,false,null,null,null,292.0,0.0,0,0,0.0,292.0
"""3/5/2025""","""Pago del pedido""","""702-2273214-7058614""","""Truper ESE-20, Escalera de ext…",3103.45,0.0,-648.0,496.55,2952.0,null,"""2025-05-02T18:23:39+00:00""","""2025-05-03T18:34:55+00:00""","""Shipped""","""Merchant""","""Amazon.com.mx""","""WebsiteOrderChannel""",null,"""Standard""","""Truper ESE-20, Escalera de ext…","""ESE-20""","""B013R4N3